# 1. Upload and importing librarys 

In [1]:
# for Colab:
#pip install scipy
#pip install -q hvplot

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats 
import hvplot.pandas

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

#from sklearn.metrics import (
#    accuracy_score, confusion_matrix, classification_report, 
#    roc_auc_score, roc_curve, auc,
#    plot_confusion_matrix, plot_roc_curve
#)
#from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

#from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

#import tensorflow as tf
#from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.metrics import AUC

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# 2. Datasets

1. French Gouverment 2013   : ``fg_2013``
2. French Gouverment 2014   : ``fg_2014``
3. EAA                      : ``EAA``

In [4]:
# Dataset French Gouv 2013
fg_2013 = pd.read_csv("Data/cl_JUIN_2013-complet3.csv", 
                      sep=';', encoding='latin-1')
fg_2013.columns


Index(['Marque', 'Modèle dossier', 'Modèle UTAC', 'Désignation commerciale',
       'CNIT', 'Type Variante Version (TVV)', 'Carburant', 'Hybride',
       'Puissance administrative', 'Puissance maximale (kW)',
       'Boîte de vitesse', 'Consommation urbaine (l/100km)',
       'Consommation extra-urbaine (l/100km)', 'Consommation mixte (l/100km)',
       'CO2 (g/km)', 'CO type I (g/km)', 'HC (g/km)', 'NOX (g/km)',
       'HC+NOX (g/km)', 'Particules (g/km)', 'masse vide euro min (kg)',
       'masse vide euro max (kg)', 'Champ V9', 'Date de mise à jour',
       'Carrosserie', 'gamme'],
      dtype='object')

In [5]:
# Dataset French Gouv 2014
fg_2014 = pd.read_csv("Data/mars-2014-complete.csv", 
                      sep=';', encoding='latin-1')

fg_2014.columns


Index(['lib_mrq', 'lib_mod_doss', 'lib_mod', 'dscom', 'cnit', 'tvv', 'cod_cbr',
       'hybride', 'puiss_admin_98', 'puiss_max', 'typ_boite_nb_rapp',
       'conso_urb', 'conso_exurb', 'conso_mixte', 'co2', 'co_typ_1', 'hc',
       'nox', 'hcnox', 'ptcl', 'masse_ordma_min', 'masse_ordma_max',
       'champ_v9', 'date_maj', 'Carrosserie', 'gamme', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29'],
      dtype='object')

## 2.1 Column Mapping Dictonary for ``fg_2013`` and `fg_2014`

In [5]:
# Create a dictionary to map column names from fg_2014 to fg_2013
column_map = dict(zip(fg_2014.columns, fg_2013.columns))

# Rename the columns in fg_2014
fg_2014 = fg_2014.rename(columns=column_map)

In [6]:
column_name_mapping = {
    'Marque': 'Brand',
    'lib_mrq': 'Brand',
    'Modèle dossier': 'Model file',
    'Modèle UTAC': 'UTAC model',
    'Désignation commerciale': 'Commercial designation',
    'CNIT': 'CNIT',
    'Type Variante Version (TVV)': 'Type Variant Version (TVV)',
    'Carburant': 'Fuel',
    'Hybride': 'Hybrid',
    'Puissance administrative': 'Administrative power',
    'Puissance maximale (kW)': 'Maximum power (kW)',
    'Boîte de vitesse': 'Transmission',
    'Consommation urbaine (l/100km)': 'Urban consumption (l/100km)',
    'Consommation extra-urbaine (l/100km)': 'Extra-urban consumption (l/100km)',
    'Consommation mixte (l/100km)': 'Combined consumption (l/100km)',
    'CO2 (g/km)': 'CO2 (g/km)',
    'CO type I (g/km)': 'CO type I (g/km)',
    'HC (g/km)': 'HC (g/km)',
    'NOX (g/km)': 'NOX (g/km)',
    'HC+NOX (g/km)': 'HC+NOX (g/km)',
    'Particules (g/km)': 'Particulates (g/km)',
    'masse vide euro min (kg)': 'Empty mass euro min (kg)',
    'masse vide euro max (kg)': 'Empty mass euro max (kg)',
    'Champ V9': 'Field V9',
    'Date de mise à jour': 'Update date',
    'Carrosserie': 'Body',
    'gamme': 'Range'
    }

In [7]:
# Rename the columns using the mapping dictionary
fg_2013.rename(columns=column_name_mapping, inplace=True)

# Rename the columns using the mapping dictionary
fg_2014.rename(columns=column_name_mapping, inplace=True)

print("fg_2013 - columns:""\n",fg_2013.columns,"\n")
print("fg_2014 - columns:""\n",fg_2014.columns)


fg_2013 - columns:
 Index(['Brand', 'Model file', 'UTAC model', 'Commercial designation', 'CNIT',
       'Type Variant Version (TVV)', 'Fuel', 'Hybrid', 'Administrative power',
       'Maximum power (kW)', 'Transmission', 'Urban consumption (l/100km)',
       'Extra-urban consumption (l/100km)', 'Combined consumption (l/100km)',
       'CO2 (g/km)', 'CO type I (g/km)', 'HC (g/km)', 'NOX (g/km)',
       'HC+NOX (g/km)', 'Particulates (g/km)', 'Empty mass euro min (kg)',
       'Empty mass euro max (kg)', 'Field V9', 'Update date', 'Body', 'Range'],
      dtype='object') 

fg_2014 - columns:
 Index(['Brand', 'Model file', 'UTAC model', 'Commercial designation', 'CNIT',
       'Type Variant Version (TVV)', 'Fuel', 'Hybrid', 'Administrative power',
       'Maximum power (kW)', 'Transmission', 'Urban consumption (l/100km)',
       'Extra-urban consumption (l/100km)', 'Combined consumption (l/100km)',
       'CO2 (g/km)', 'CO type I (g/km)', 'HC (g/km)', 'NOX (g/km)',
       'HC+NOX (g/km)',

In [8]:
print("2013 shape:\n",fg_2013.shape)
print("2014 shape:\n",fg_2014.shape)

2013 shape:
 (44850, 26)
2014 shape:
 (55044, 30)


Columns are "normalized", let's have a look to the ``Unnamed`` columns.

In [9]:
fg_2014[['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29']].head()

print(fg_2014[['Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29']].describe()
)

       Unnamed: 26  Unnamed: 27  Unnamed: 28  Unnamed: 29
count         0.00         0.00         0.00         0.00
mean           NaN          NaN          NaN          NaN
std            NaN          NaN          NaN          NaN
min            NaN          NaN          NaN          NaN
25%            NaN          NaN          NaN          NaN
50%            NaN          NaN          NaN          NaN
75%            NaN          NaN          NaN          NaN
max            NaN          NaN          NaN          NaN


### D1 - Descision 1 (Delete 'Unnamed: 26' - 'Unnamed: 29')
No Values in the List -> Delete them

In [14]:
fg_2014 = fg_2014.drop(fg_2014.columns[-4:], axis=1)

fg_2014.shape

(55044, 26)

## Dataset EAA_EU

In [15]:
# Dataset EAA EU

EAA_EU = pd.read_csv("data/data.csv")

display(EAA_EU.head())

C:\Users\Stefan\AppData\Local\Temp\ipykernel_7440\2883326208.py:3: DtypeWarning: Columns (2,3,8,10,12,28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  EAA_EU = pd.read_csv("data/data.csv")


,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,Ve,Mk,Cn,Ct,Cr,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),Ft,Fm,ec (cm3),ep (KW),z (Wh/km),IT,Ernedc (g/km),Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,Electric range (km)
0,80960000,DE,IP-MLB58_Z_B0_0667-WAU-1,VOLKSWAGEN,AUDI AG,AUDI AG,NaN,e1*2007/46*1801*33,F2,ADMKDQ2,QA8QA8D5024051C21S54BIBEM1H66FLD2,AUDI,S6 AVANT,M1,M1,1,2095.00,2237.00,NaN,191.00,2931.00,1603.00,1618.00,DIESEL,H,2967.00,253.00,NaN,NaN,NaN,NaN,NaN,NaN,P,2022,2022-08-26,7.30,NaN
1,80960001,DE,IP-MLB58_Z_A0_0659-WAU-1,VOLKSWAGEN,AUDI AG,AUDI AG,NaN,e1*2007/46*1801*33,F2,LDMGAQ1,QA8QA8D5023031B18S54BIAEM1H56FLD2,AUDI,A6 LIMOUSINE,M1,M1,1,1910.00,2112.00,NaN,179.00,2931.00,1603.00,1618.00,DIESEL,H,2967.00,210.00,NaN,NaN,NaN,NaN,NaN,NaN,P,2022,2022-04-04,6.80,NaN
2,80960002,DE,IP-MLB58_Z_A0_0689-WAU-1,VOLKSWAGEN,AUDI AG,AUDI AG,NaN,e1*2007/46*1801*33,F2,ADTPAF1,FD7FD7CK001031B19S54BIAEM1H46FSD1,AUDI,A6 AVANT,M1,M1,1,1780.00,1932.00,NaN,149.00,2931.00,1603.00,1618.00,DIESEL,H,1968.00,150.00,NaN,NaN,NaN,NaN,NaN,NaN,P,2022,2022-04-11,5.70,NaN
3,80960003,DE,IP-MLB58_Z_C0_0681-WAU-1,VOLKSWAGEN,AUDI AG,AUDI AG,NaN,e1*2007/46*1801*33,F2,ADRYAQ1,QD7AD7DK001101B20S54BICEM9H66FSB3,AUDI,A6 AVANT 55 TFSI E,M1,M1,1,2150.00,2333.00,NaN,35.00,2931.00,1603.00,1618.00,PETROL/ELECTRIC,P,1984.00,195.00,198.00,NaN,NaN,NaN,NaN,NaN,P,2022,2022-03-15,1.50,83.00
4,80960004,DE,IP-MLB58_Z_A0_0689-WAU-1,VOLKSWAGEN,AUDI AG,AUDI AG,NaN,e1*2007/46*1801*33,F2,ADTPAF1,FD7FD7CK001031B18S54BIAEM1H46FSD1,AUDI,A6 AVANT,M1,M1,1,1780.00,1935.00,NaN,148.00,2931.00,1603.00,1618.00,DIESEL,H,1968.00,150.00,NaN,NaN,NaN,NaN,NaN,NaN,P,2022,2022-04-22,5.70,NaN


### Reduce Dataset for developing the coding



In [19]:
EAA_EU_sample = EAA_EU.sample(n=1000, random_state=42)
EAA_EU_sample.shape

(1000, 38)

## 3. 🔍 Exploratory Data Analysis

> **OVERALL GOAL:** 
    > - Get an understanding for which variables are important, view summary statistics, and visualize the data

In [20]:
EAA_EU.columns


Index(['ID', 'Country', 'VFN', 'Mp', 'Mh', 'Man', 'MMS', 'Tan', 'T', 'Va',
       'Ve', 'Mk', 'Cn', 'Ct', 'Cr', 'r', 'm (kg)', 'Mt', 'Enedc (g/km)',
       'Ewltp (g/km)', 'W (mm)', 'At1 (mm)', 'At2 (mm)', 'Ft', 'Fm',
       'ec (cm3)', 'ep (KW)', 'z (Wh/km)', 'IT', 'Ernedc (g/km)',
       'Erwltp (g/km)', 'De', 'Vf', 'Status', 'year', 'Date of registration',
       'Fuel consumption ', 'Electric range (km)'],
      dtype='object')

In [21]:
EAA_EU.describe()

,ID,MMS,r,m (kg),Mt,Enedc (g/km),Ewltp (g/km),W (mm),At1 (mm),At2 (mm),ec (cm3),ep (KW),z (Wh/km),Ernedc (g/km),Erwltp (g/km),De,Vf,year,Fuel consumption,Electric range (km)
count,9479280.00,0.00,9479280.00,9479149.00,8418686.00,1528918.00,9464562.00,9457478.00,9266098.00,9250258.00,8201472.00,9455287.00,2083147.00,0.00,5067277.00,0.00,0.00,9479280.00,7244541.00,1612374.00
mean,79871081.88,NaN,1.00,1528.41,1649.68,5.84,108.37,2670.37,1553.39,1555.46,1544.10,109.49,172.00,NaN,1.52,NaN,NaN,2022.00,5.29,250.75
std,3955287.95,NaN,0.00,352.99,363.93,13.43,58.27,168.17,59.32,63.52,518.67,57.07,31.46,NaN,0.49,NaN,NaN,0.00,1.77,187.77
min,13719023.00,NaN,1.00,570.00,611.00,0.00,0.00,600.00,520.00,520.00,4.00,4.00,10.00,NaN,0.50,NaN,NaN,2022.00,0.50,13.00
25%,77254054.75,NaN,1.00,1263.00,1381.00,0.00,101.00,2570.00,1520.00,1520.00,1197.00,74.00,154.00,NaN,1.10,NaN,NaN,2022.00,4.70,62.00
50%,80041193.50,NaN,1.00,1457.00,1582.00,0.00,123.00,2667.00,1559.00,1558.00,1498.00,96.00,165.00,NaN,1.70,NaN,NaN,2022.00,5.40,230.00
75%,83288707.25,NaN,1.00,1739.00,1866.00,0.00,140.00,2730.00,1590.00,1595.00,1968.00,129.00,183.00,NaN,2.00,NaN,NaN,2022.00,6.00,431.00
max,85873927.00,NaN,1.00,4600.00,4711.00,49.00,730.00,5405.00,3000.00,3000.00,7993.00,1177.00,999.00,NaN,6.00,NaN,NaN,2022.00,28.00,843.00


In [22]:
EAA_EU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9479280 entries, 0 to 9479279
Data columns (total 38 columns):
 #   Column                Dtype  
---  ------                -----  
 0   ID                    int64  
 1   Country               object 
 2   VFN                   object 
 3   Mp                    object 
 4   Mh                    object 
 5   Man                   object 
 6   MMS                   float64
 7   Tan                   object 
 8   T                     object 
 9   Va                    object 
 10  Ve                    object 
 11  Mk                    object 
 12  Cn                    object 
 13  Ct                    object 
 14  Cr                    object 
 15  r                     int64  
 16  m (kg)                float64
 17  Mt                    float64
 18  Enedc (g/km)          float64
 19  Ewltp (g/km)          float64
 20  W (mm)                float64
 21  At1 (mm)              float64
 22  At2 (mm)              float64
 23  Ft     

## 3.1 Missing Values

In [23]:
EAA_EU.count()

ID                      9479280
Country                 9479280
VFN                     8655157
Mp                      8860902
Mh                      9479280
Man                     9479279
MMS                           0
Tan                     9456297
T                       9475953
Va                      9460466
Ve                      9431846
Mk                      9478878
Cn                      9335123
Ct                      9464260
Cr                      9479280
r                       9479280
m (kg)                  9479149
Mt                      8418686
Enedc (g/km)            1528918
Ewltp (g/km)            9464562
W (mm)                  9457478
At1 (mm)                9266098
At2 (mm)                9250258
Ft                      9479280
Fm                      9479251
ec (cm3)                8201472
ep (KW)                 9455287
z (Wh/km)               2083147
IT                      5892595
Ernedc (g/km)                 0
Erwltp (g/km)           5067277
De      

In [24]:
EAA_EU.sum().isna()

In [1]:
# missing values in [%]
missing_percent = (eaa_eu.isnull().sum() / len(eaa_eu)) * 100

# show all NA over 80%
columns_with_high_missing = missing_percent[missing_percent > 80]

# Display missing Values > 80%
print("Spalten mit mehr als 80% fehlenden Werten:")
print(columns_with_high_missing)

NameError: name 'eaa_eu' is not defined

In [ ]:
# Create a heatmap of missing values
sns.heatmap(df.isnull(), cbar=False)